In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json /content/
!mv /content/kaggle.json ~/.kaggle/
!chmod 600 /content/.kaggle/kaggle.json

!kaggle datasets download chaitu20/ipl-dataset2008-2025
!unzip /content/ipl-dataset2008-2025.zip -d /content/dataset
import os
os.rename('/content/dataset/IPL.csv', '/content/dataset/data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists
chmod: cannot access '/content/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/chaitu20/ipl-dataset2008-2025
License(s): CC0-1.0
ipl-dataset2008-2025.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/ipl-dataset2008-2025.zip
  inflating: /content/dataset/IPL.csv  


In [ ]:
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/dataset/data.csv')
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
print(df.info())

/tmp/ipython-input-459583345.py:7: DtypeWarning: Columns (28,29,30,31,43,46,47,48,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/dataset/data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278205 entries, 0 to 278204
Data columns (total 64 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Unnamed: 0         278205 non-null  int64         
 1   match_id           278205 non-null  int64         
 2   date               278205 non-null  datetime64[ns]
 3   match_type         278205 non-null  object        
 4   event_name         278205 non-null  object        
 5   innings            278205 non-null  int64         
 6   batting_team       278205 non-null  object        
 7   bowling_team       278205 non-null  object        
 8   over               278205 non-null  int64         
 9   ball               278205 non-null  int64         
 10  ball_no            278205 non-null  float64       
 11  batter             278205 non-null  object        
 12  bat_pos            278205 non-null  int64         
 13  runs_batter        278205 non-null  int64   

In [ ]:
df['batting_team'] = df['batting_team'].replace({'Delhi Daredevils':'Delhi Capitals', 'Kings XI Punjab':'Punjab Kings', 'Royal Challengers Bangalore':'Royal Challengers Bengaluru'})

In [ ]:
dfunct_teams = ['Deccan Chargers', 'Gujarat Lions', 'Kochi Tuskers Kerala', 'Rising Pune Supergiant', 'Pune Warriors India']
df = df[~df['batting_team'].isin(dfunct_teams)]
df = df[~df['bowling_team'].isin(dfunct_teams)]

In [ ]:
print(df.columns)
for i in range(64):
  print(df.columns[i])
  print()
  print(df.iloc[:,i].describe())
  print('\n\n')

Index(['Unnamed: 0', 'match_id', 'date', 'match_type', 'event_name', 'innings',
       'batting_team', 'bowling_team', 'over', 'ball', 'ball_no', 'batter',
       'bat_pos', 'runs_batter', 'balls_faced', 'bowler', 'valid_ball',
       'runs_extras', 'runs_total', 'runs_bowler', 'runs_not_boundary',
       'extra_type', 'non_striker', 'non_striker_pos', 'wicket_kind',
       'player_out', 'fielders', 'runs_target', 'review_batter',
       'team_reviewed', 'review_decision', 'umpire', 'umpires_call',
       'player_of_match', 'match_won_by', 'win_outcome', 'toss_winner',
       'toss_decision', 'venue', 'city', 'day', 'month', 'year', 'season',
       'gender', 'team_type', 'superover_winner', 'result_type', 'method',
       'balls_per_over', 'overs', 'event_match_no', 'stage', 'match_number',
       'team_runs', 'team_balls', 'team_wicket', 'new_batter', 'batter_runs',
       'batter_balls', 'bowler_wicket', 'batting_partners', 'next_batter',
       'striker_out'],
      dtype='object')

In [ ]:
to_remove = ['Unnamed: 0', 'event_name', 'match_type', 'win_outcome', 'day', 'month','year', 'team_type', 'overs',
             'balls_per_over', 'superover_winner', 'umpire', 'player_of_match','city', 'gender', 'result_type',
             'method', 'match_number', 'next_batter', 'review_decision', 'team_reviewed', 'review_batter']
print(len(to_remove))
df=df.drop(to_remove, axis=1)

22


In [ ]:
def func(row):
  if row['toss_decision'] == 'bat':
     return row['toss_winner']
  else:
    if row['toss_winner'] == row['batting_team']:
      return row['bowling_team']
    else:
      return row['batting_team']
df['first_to_bat'] = df.apply(func, axis=1)

In [ ]:
df = df.drop(['toss_decision' ,'toss_winner'], axis=1)


In [ ]:
print(df.columns)


Index(['match_id', 'date', 'innings', 'batting_team', 'bowling_team', 'over',
       'ball', 'ball_no', 'batter', 'bat_pos', 'runs_batter', 'balls_faced',
       'bowler', 'valid_ball', 'runs_extras', 'runs_total', 'runs_bowler',
       'runs_not_boundary', 'extra_type', 'non_striker', 'non_striker_pos',
       'wicket_kind', 'player_out', 'fielders', 'runs_target', 'umpires_call',
       'match_won_by', 'venue', 'season', 'event_match_no', 'stage',
       'team_runs', 'team_balls', 'team_wicket', 'new_batter', 'batter_runs',
       'batter_balls', 'bowler_wicket', 'batting_partners', 'striker_out',
       'first_to_bat'],
      dtype='object')


In [ ]:
matches = df.drop(['over', 'ball', 'ball_no','batter', 'bat_pos', 'runs_batter',
                   'balls_faced','bowler', 'valid_ball', 'runs_extras',
                   'runs_total', 'runs_bowler', 'runs_not_boundary', 'extra_type',
                   'non_striker', 'non_striker_pos','wicket_kind', 'player_out',
                   'fielders', 'umpires_call', 'event_match_no','new_batter',
                   'batter_runs', 'batter_balls', 'bowler_wicket', 'batting_partners',
                   'striker_out'], axis=1).reset_index(drop=True)
matches['stage'] = matches['stage'].replace({'Unknown': 'league'})

In [ ]:
matches_inning_wise = matches.groupby(['match_id', 'innings']).agg({
    'date':'first'
    ,'venue':'first'
    ,'batting_team':'first'
    ,'bowling_team':'first'
    ,'first_to_bat':'first'
    ,'stage':'first'
    ,'runs_target':'first'
    ,'match_won_by':'first'
    , 'team_runs' : 'last'
    , 'team_balls' :	'last'
    , "team_wicket"	: 'last'

})
matches_inning_wise = matches_inning_wise.reset_index()
matches_inning_wise.head(5)

,match_id,innings,date,venue,batting_team,bowling_team,first_to_bat,stage,runs_target,match_won_by,team_runs,team_balls,team_wicket
0,335982,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,league,NaN,Kolkata Knight Riders,222,120,3
1,335982,2,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bengaluru,Kolkata Knight Riders,Royal Challengers Bengaluru,league,223.0,Kolkata Knight Riders,82,91,10
2,335983,1,2008-04-19,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,league,NaN,Chennai Super Kings,240,120,5
3,335983,2,2008-04-19,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Chennai Super Kings,Chennai Super Kings,league,241.0,Chennai Super Kings,207,120,4
4,335984,1,2008-04-19,Feroz Shah Kotla,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,league,NaN,Delhi Daredevils,129,120,8


In [ ]:
matches_sumary = matches_inning_wise.groupby('match_id').agg({
    'match_id':'first'
    ,'date':'first'
    ,'venue':'first'
    ,'batting_team':'first'
    ,'bowling_team':'first'
    ,'first_to_bat':'first'
    ,'stage':'first'
    ,'runs_target':'last'
    ,'match_won_by':'first'
})
matches_sumary= matches_sumary.rename(columns={'batting_team':'team_a', 'bowling_team': 'team_b'}).reset_index(drop=True)
matches_sumary['team_a_runs'] = matches_inning_wise.groupby('match_id').agg({'team_runs':'first'})
matches_sumary['team_b_runs'] = matches_inning_wise.groupby('match_id').agg({'team_runs':'last'})
matches_sumary['team_a_wicket'] = matches_inning_wise.groupby('match_id').agg({'team_wicket':'first'})
matches_sumary['team_b_wicket'] = matches_inning_wise.groupby('match_id').agg({'team_wicket':'last'})
matches_sumary['team_a_balls'] = matches_inning_wise.groupby('match_id').agg({'team_balls':'first'})
matches_sumary['team_b_balls'] = matches_inning_wise.groupby('match_id').agg({'team_balls':'last'})

matches_sumary.head(5)

,match_id,date,venue,team_a,team_b,first_to_bat,stage,runs_target,match_won_by,team_a_runs,team_b_runs,team_a_wicket,team_b_wicket,team_a_balls,team_b_balls
0,335982,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,league,223.0,Kolkata Knight Riders,NaN,NaN,NaN,NaN,NaN,NaN
1,335983,2008-04-19,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,league,241.0,Chennai Super Kings,NaN,NaN,NaN,NaN,NaN,NaN
2,335984,2008-04-19,Feroz Shah Kotla,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,league,130.0,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN
3,335985,2008-04-20,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,league,166.0,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN
4,335987,2008-04-21,Sawai Mansingh Stadium,Punjab Kings,Rajasthan Royals,Kings XI Punjab,league,167.0,Rajasthan Royals,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
innings_1_summary = matches_inning_wise[matches_inning_wise['innings'] == 1]
innings_1_summary = innings_1_summary.drop(['runs_target'], axis =1).reset_index()
innings_2_summary = matches_inning_wise[matches_inning_wise['innings'] == 2].reset_index()

In [ ]:
innings_2_summary

,index,match_id,innings,date,venue,batting_team,bowling_team,first_to_bat,stage,runs_target,match_won_by,team_runs,team_balls,team_wicket
0,1,335982,2,2008-04-18,M Chinnaswamy Stadium,Royal Challengers Bengaluru,Kolkata Knight Riders,Royal Challengers Bengaluru,league,223.0,Kolkata Knight Riders,82,91,10
1,3,335983,2,2008-04-19,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Chennai Super Kings,Chennai Super Kings,league,241.0,Chennai Super Kings,207,120,4
2,5,335984,2,2008-04-19,Feroz Shah Kotla,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,league,130.0,Delhi Daredevils,132,91,1
3,7,335985,2,2008-04-20,Wankhede Stadium,Royal Challengers Bengaluru,Mumbai Indians,Mumbai Indians,league,166.0,Royal Challengers Bangalore,166,118,5
4,9,335987,2,2008-04-21,Sawai Mansingh Stadium,Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,league,167.0,Rajasthan Royals,168,109,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,2090,1473508,2,2025-05-29,Maharaja Yadavindra Singh International Cricke...,Royal Challengers Bengaluru,Punjab Kings,Punjab Kings,Qualifier 1,102.0,Royal Challengers Bengaluru,106,60,2
1028,2092,1473509,2,2025-05-30,Maharaja Yadavindra Singh International Cricke...,Gujarat Titans,Mumbai Indians,Mumbai Indians,Eliminator,229.0,Mumbai Indians,208,120,6
1029,2094,1473510,2,2025-06-01,"Narendra Modi Stadium, Ahmedabad",Punjab Kings,Mumbai Indians,Mumbai Indians,Qualifier 2,204.0,Punjab Kings,207,114,5
1030,2096,1473511,2,2025-06-03,"Narendra Modi Stadium, Ahmedabad",Punjab Kings,Royal Challengers Bengaluru,Royal Challengers Bengaluru,Final,191.0,Royal Challengers Bengaluru,184,120,7


In [ ]:
innings_1_summary

,index,match_id,innings,date,venue,batting_team,bowling_team,first_to_bat,stage,match_won_by,team_runs,team_balls,team_wicket
0,0,335982,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,league,Kolkata Knight Riders,222,120,3
1,2,335983,1,2008-04-19,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,league,Chennai Super Kings,240,120,5
2,4,335984,1,2008-04-19,Feroz Shah Kotla,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,league,Delhi Daredevils,129,120,8
3,6,335985,1,2008-04-20,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,league,Royal Challengers Bangalore,165,120,6
4,8,335987,1,2008-04-21,Sawai Mansingh Stadium,Punjab Kings,Rajasthan Royals,Kings XI Punjab,league,Rajasthan Royals,166,119,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,2089,1473508,1,2025-05-29,Maharaja Yadavindra Singh International Cricke...,Punjab Kings,Royal Challengers Bengaluru,Punjab Kings,Qualifier 1,Royal Challengers Bengaluru,101,85,10
1033,2091,1473509,1,2025-05-30,Maharaja Yadavindra Singh International Cricke...,Mumbai Indians,Gujarat Titans,Mumbai Indians,Eliminator,Mumbai Indians,228,120,5
1034,2093,1473510,1,2025-06-01,"Narendra Modi Stadium, Ahmedabad",Mumbai Indians,Punjab Kings,Mumbai Indians,Qualifier 2,Punjab Kings,203,120,6
1035,2095,1473511,1,2025-06-03,"Narendra Modi Stadium, Ahmedabad",Royal Challengers Bengaluru,Punjab Kings,Royal Challengers Bengaluru,Final,Royal Challengers Bengaluru,190,120,9


In [ ]:
data = df.drop(['over', 'ball','umpires_call',
        'event_match_no', 'striker_out', 'batting_partners', 'umpires_call', 'fielders',], axis =1 )
data.columns

Index(['match_id', 'date', 'innings', 'batting_team', 'bowling_team',
       'ball_no', 'batter', 'bat_pos', 'runs_batter', 'balls_faced', 'bowler',
       'valid_ball', 'runs_extras', 'runs_total', 'runs_bowler',
       'runs_not_boundary', 'extra_type', 'non_striker', 'non_striker_pos',
       'wicket_kind', 'player_out', 'runs_target', 'match_won_by', 'venue',
       'season', 'stage', 'team_runs', 'team_balls', 'team_wicket',
       'new_batter', 'batter_runs', 'batter_balls', 'bowler_wicket',
       'first_to_bat'],
      dtype='object')

In [ ]:
batters_matchwise = data.groupby(["match_id","batter"]).agg({
    'batter':'first',
    'runs_batter':'last',
    'balls_faced':'last',
})
batters_matchwise = batters_matchwise.reset_index(drop=True)
batters = batters_matchwise.groupby(['batter']).agg({
        'batter':'first',
    'runs_batter':'sum',
    'balls_faced':'sum'
})
batters['batting_avg'] = batters['runs_batter']/batters['balls_faced']
batters = batters.reset_index(drop=True)
print(batters)

bowlers_matchwise = data.groupby(["match_id","bowler"]).agg({
   'bowler':'first',
    'runs_bowler':'sum',
    'bowler_wicket':'sum'
})
def count_bowlers(val):
  return data[data['bowler'] == val].shape[0]
bowlers_matchwise = bowlers_matchwise.reset_index(drop=True)
bowlers_matchwise["balls_bowled"] = bowlers_matchwise["bowler"].apply(count_bowlers)
bowlers = bowlers_matchwise.groupby(['bowler']).agg({
        'bowler':'first',
    'runs_bowler':'sum',
    'balls_bowled':'first'
})
bowlers['economy'] = bowlers['runs_bowler']*6/bowlers['balls_bowled']
bowlers = bowlers.reset_index(drop=True)
print(bowlers)

             batter  runs_batter  balls_faced  batting_avg
0    A Ashish Reddy            8           18     0.444444
1          A Badoni           25           46     0.543478
2        A Chandila            1            1     1.000000
3          A Chopra            1            4     0.250000
4       A Choudhary            0            2     0.000000
..              ...          ...          ...          ...
667   Yashpal Singh            0            4     0.000000
668     Younis Khan            0            1     0.000000
669   Yudhvir Singh            1            4     0.250000
670    Yuvraj Singh           49          114     0.429825
671          Z Khan           17           24     0.708333

[672 rows x 4 columns]
             bowler  runs_bowler  balls_bowled    economy
0    A Ashish Reddy          159           101   9.445545
1          A Badoni           49            35   8.400000
2        A Chandila          222           210   6.342857
3       A Choudhary          123    

In [ ]:
bowlers_matchwise.sort_values(by = 'balls_bowled', ascending=False)

,bowler,runs_bowler,bowler_wicket,balls_bowled
11820,R Ashwin,25,0,4634
11664,R Ashwin,30,0,4634
8000,R Ashwin,27,1,4634
7566,R Ashwin,30,0,4634
5081,R Ashwin,7,1,4634
...,...,...,...,...
7001,SN Khan,6,0,2
11451,Ishan Kishan,4,0,1
8910,YBK Jaiswal,6,0,1
11313,Atharva Taide,4,0,1


In [ ]:
data[data['bowler'] == 'Yuvraj Singh'].shape[0]

743

In [ ]:
df['wicket_kind'] = df['wicket_kind'].fillna('not_out')

In [ ]:
df['wicket_kind'].value_counts()

,count
wicket_kind,
not_out,234471
caught,7744
bowled,2044
run out,991
lbw,753
caught and bowled,340
stumped,322
retired hurt,16
hit wicket,16


In [ ]:
df

,match_id,date,innings,batting_team,bowling_team,over,ball,ball_no,batter,bat_pos,...,team_runs,team_balls,team_wicket,new_batter,batter_runs,batter_balls,bowler_wicket,batting_partners,striker_out,first_to_bat
0,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,0.1,SC Ganguly,1,...,1,1,0,NaN,0,1,0,"('BB McCullum', 'SC Ganguly')",False,Kolkata Knight Riders
1,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,0.2,BB McCullum,2,...,1,2,0,NaN,0,1,0,"('BB McCullum', 'SC Ganguly')",False,Kolkata Knight Riders
2,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,0.3,BB McCullum,2,...,2,2,0,NaN,0,1,0,"('BB McCullum', 'SC Ganguly')",False,Kolkata Knight Riders
3,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,0.3,BB McCullum,2,...,2,3,0,NaN,0,2,0,"('BB McCullum', 'SC Ganguly')",False,Kolkata Knight Riders
4,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,0.4,BB McCullum,2,...,2,4,0,NaN,0,3,0,"('BB McCullum', 'SC Ganguly')",False,Kolkata Knight Riders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278200,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19,2,19.2,Shashank Singh,6,...,162,116,7,NaN,39,26,0,"('KA Jamieson', 'Shashank Singh')",False,Royal Challengers Bengaluru
278201,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19,3,19.3,Shashank Singh,6,...,168,117,7,NaN,45,27,0,"('KA Jamieson', 'Shashank Singh')",False,Royal Challengers Bengaluru
278202,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19,4,19.4,Shashank Singh,6,...,172,118,7,NaN,49,28,0,"('KA Jamieson', 'Shashank Singh')",False,Royal Challengers Bengaluru
278203,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19,5,19.5,Shashank Singh,6,...,178,119,7,NaN,55,29,0,"('KA Jamieson', 'Shashank Singh')",False,Royal Challengers Bengaluru


In [ ]:
def set_econ(row):
  # Get the economy for the current bowler, ensuring a scalar is returned
  econ_series = bowlers[bowlers['bowler'] == row['bowler']]['economy']
  # Check if the series is not empty to prevent IndexError
  return econ_series.iloc[0] if not econ_series.empty else np.nan
def batter_avg(row):
  # Get the batting average for the current batter, ensuring a scalar is returned
  avg_series = batters[batters['batter'] == row['batter']]['batting_avg']
  # Check if the series is not empty to prevent IndexError
  return avg_series.iloc[0] if not avg_series.empty else np.nan

data['batter_avg'] = data.apply(batter_avg, axis=1)
data['econ'] = data.apply(set_econ, axis=1)

In [ ]:
data


,match_id,date,innings,batting_team,bowling_team,ball_no,batter,bat_pos,runs_batter,balls_faced,...,team_runs,team_balls,team_wicket,new_batter,batter_runs,batter_balls,bowler_wicket,first_to_bat,batter_avg,econ
0,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.1,SC Ganguly,1,0,1,...,1,1,0,NaN,0,1,0,Kolkata Knight Riders,0.043478,7.301829
1,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.2,BB McCullum,2,0,1,...,1,2,0,NaN,0,1,0,Kolkata Knight Riders,0.276923,7.301829
2,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,BB McCullum,2,0,0,...,2,2,0,NaN,0,1,0,Kolkata Knight Riders,0.276923,7.301829
3,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,BB McCullum,2,0,1,...,2,3,0,NaN,0,2,0,Kolkata Knight Riders,0.276923,7.301829
4,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.4,BB McCullum,2,0,1,...,2,4,0,NaN,0,3,0,Kolkata Knight Riders,0.276923,7.301829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278200,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.2,Shashank Singh,6,0,1,...,162,116,7,NaN,39,26,0,Royal Challengers Bengaluru,1.181818,8.044843
278201,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.3,Shashank Singh,6,6,1,...,168,117,7,NaN,45,27,0,Royal Challengers Bengaluru,1.181818,8.044843
278202,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.4,Shashank Singh,6,4,1,...,172,118,7,NaN,49,28,0,Royal Challengers Bengaluru,1.181818,8.044843
278203,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.5,Shashank Singh,6,6,1,...,178,119,7,NaN,55,29,0,Royal Challengers Bengaluru,1.181818,8.044843


In [ ]:
data['runs_bowler'].value_counts()

,count
runs_bowler,
1,99050
0,88984
4,28455
2,15990
6,12837
3,821
5,482
7,86


In [ ]:
data['stage'] = data['stage'].replace({'Unknown': 'league'})
data

,match_id,date,innings,batting_team,bowling_team,ball_no,batter,bat_pos,runs_batter,balls_faced,...,team_runs,team_balls,team_wicket,new_batter,batter_runs,batter_balls,bowler_wicket,first_to_bat,batter_avg,econ
0,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.1,SC Ganguly,1,0,1,...,1,1,0,NaN,0,1,0,Kolkata Knight Riders,0.043478,7.301829
1,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.2,BB McCullum,2,0,1,...,1,2,0,NaN,0,1,0,Kolkata Knight Riders,0.276923,7.301829
2,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,BB McCullum,2,0,0,...,2,2,0,NaN,0,1,0,Kolkata Knight Riders,0.276923,7.301829
3,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,BB McCullum,2,0,1,...,2,3,0,NaN,0,2,0,Kolkata Knight Riders,0.276923,7.301829
4,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.4,BB McCullum,2,0,1,...,2,4,0,NaN,0,3,0,Kolkata Knight Riders,0.276923,7.301829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278200,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.2,Shashank Singh,6,0,1,...,162,116,7,NaN,39,26,0,Royal Challengers Bengaluru,1.181818,8.044843
278201,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.3,Shashank Singh,6,6,1,...,168,117,7,NaN,45,27,0,Royal Challengers Bengaluru,1.181818,8.044843
278202,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.4,Shashank Singh,6,4,1,...,172,118,7,NaN,49,28,0,Royal Challengers Bengaluru,1.181818,8.044843
278203,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.5,Shashank Singh,6,6,1,...,178,119,7,NaN,55,29,0,Royal Challengers Bengaluru,1.181818,8.044843


In [ ]:
to_drop = [ 'runs_bowler',
           'wicket_kind', 'new_batter', ]
data = data.drop(to_drop, axis =1)
data

,match_id,date,innings,batting_team,bowling_team,ball_no,batter,bat_pos,runs_batter,balls_faced,...,stage,team_runs,team_balls,team_wicket,batter_runs,batter_balls,bowler_wicket,first_to_bat,batter_avg,econ
0,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.1,SC Ganguly,1,0,1,...,league,1,1,0,0,1,0,Kolkata Knight Riders,0.043478,7.301829
1,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.2,BB McCullum,2,0,1,...,league,1,2,0,0,1,0,Kolkata Knight Riders,0.276923,7.301829
2,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,BB McCullum,2,0,0,...,league,2,2,0,0,1,0,Kolkata Knight Riders,0.276923,7.301829
3,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.3,BB McCullum,2,0,1,...,league,2,3,0,0,2,0,Kolkata Knight Riders,0.276923,7.301829
4,335982,2008-04-18,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.4,BB McCullum,2,0,1,...,league,2,4,0,0,3,0,Kolkata Knight Riders,0.276923,7.301829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278200,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.2,Shashank Singh,6,0,1,...,Final,162,116,7,39,26,0,Royal Challengers Bengaluru,1.181818,8.044843
278201,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.3,Shashank Singh,6,6,1,...,Final,168,117,7,45,27,0,Royal Challengers Bengaluru,1.181818,8.044843
278202,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.4,Shashank Singh,6,4,1,...,Final,172,118,7,49,28,0,Royal Challengers Bengaluru,1.181818,8.044843
278203,1473511,2025-06-03,2,Punjab Kings,Royal Challengers Bengaluru,19.5,Shashank Singh,6,6,1,...,Final,178,119,7,55,29,0,Royal Challengers Bengaluru,1.181818,8.044843


In [ ]:
data['team_runs'].value_counts()

,count
team_runs,
1,2769
0,2557
4,2121
2,2051
5,2006
...,...
273,1
280,1
282,1


In [ ]:
def is_wicket(row):
  if row['wicket_kind'] == 'not_out':
    return 0
  else:
    return 1
data['is_wicket'] = df.apply(is_wicket, axis=1)


In [ ]:
data["runs_last_6"] = (
    df.groupby(['match_id', 'innings'])["runs_total"]
      .shift(0)
      .rolling(6)
      .sum()
      .fillna(0)
)
data["runs_last_12"] = (
    df.groupby(['match_id', 'innings'])["runs_total"]
      .shift(0)
      .rolling(12)
      .sum()
      .fillna(0)
)
data["wkts_last_12"] = (
    data.groupby(['match_id', 'innings'])["is_wicket"]
      .shift(0)
      .rolling(12)
      .sum()
      .fillna(0)
)


In [ ]:
data.columns

Index(['match_id', 'date', 'innings', 'batting_team', 'bowling_team',
       'ball_no', 'batter', 'bat_pos', 'runs_batter', 'balls_faced', 'bowler',
       'valid_ball', 'runs_extras', 'runs_total', 'runs_not_boundary',
       'extra_type', 'non_striker', 'non_striker_pos', 'player_out',
       'runs_target', 'match_won_by', 'venue', 'season', 'stage', 'team_runs',
       'team_balls', 'team_wicket', 'batter_runs', 'batter_balls',
       'bowler_wicket', 'first_to_bat', 'batter_avg', 'econ', 'is_wicket',
       'runs_last_6', 'runs_last_12', 'wkts_last_12'],
      dtype='object')

In [ ]:
innings_1 = data[data['innings']==1]
def target(match_id):
  return matches_sumary[matches_sumary['match_id'] == match_id]['runs_target'].iloc[0]
innings_1['target'] = innings_1['match_id'].apply(target)
innings_2 = data[data['innings']==2]

/tmp/ipython-input-3438686711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  innings_1['target'] = innings_1['match_id'].apply(target)


In [ ]:
innings_1.to_csv('/content/dataset/innings_1.csv', index=False)
innings_2.to_csv('/content/dataset/innings_2.csv', index=False)
data.to_csv('/content/dataset/data_filtered.csv', index=False)
batters_matchwise.to_csv('/content/dataset/batters_matchwise.csv', index=False)
batters.to_csv('/content/dataset/batters.csv', index=False)
bowlers_matchwise.to_csv('/content/dataset/bowlers_matchwise.csv', index=False)
bowlers.to_csv('/content/dataset/bowlers.csv', index=False)
matches_inning_wise.to_csv('/content/dataset/matches_inning_wise.csv', index=False)
matches_sumary.to_csv('/content/dataset/matches_sumary.csv', index=False)
matches.to_csv('/content/dataset/matches.csv', index=False)

In [ ]:
!zip -r dataset.zip dataset/

  adding: dataset/ (stored 0%)
  adding: dataset/matches_sumary.csv (deflated 90%)
  adding: dataset/innings_1.csv (deflated 92%)
  adding: dataset/bowlers_matchwise.csv (deflated 75%)
  adding: dataset/data.csv (deflated 94%)
  adding: dataset/matches.csv (deflated 97%)
  adding: dataset/batters_matchwise.csv (deflated 86%)
  adding: dataset/innings_2.csv (deflated 91%)
  adding: dataset/matches_inning_wise.csv (deflated 89%)
  adding: dataset/batters.csv (deflated 57%)
  adding: dataset/bowlers.csv (deflated 48%)
  adding: dataset/data_filtered.csv (deflated 91%)


In [ ]:
from google.colab import files
files.download('/content/dataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>